import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # 加入这行代码可以只使用cpu，对于自定义LSTM参数类训练，只使用cpu训练速度更快

In [1]:
import pandas as pd
import jieba
import re
import numpy as np
import gensim
from keras_self_attention import SeqSelfAttention
import pickle
from keras.utils.np_utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [2]:
maxlen=100 #  在100个词后截断评论
max_words=50000 # 考虑前50000个出现的词 发现有321713个word_index，故调整为50000
embedding_dim=200

import keras
keras.backend.clear_session()

import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.8

strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])
print('Number of devices: %d' % strategy.num_replicas_in_sync)  # 输出设备数量

In [ ]:
def word_cut(comments):
     words = jieba.cut(comments)
     return ','.join(words)# 以逗号连接分割后的词
#apply函数默认会按行应用word_cut函数

# 从txt文件中读取停止词
def Get_stopwords(path):
    stopwords=[]
    with open(path,encoding='utf-8') as words:
        stopwords.extend([i.strip() for i in words.readlines()])
    return stopwords
stopwords=Get_stopwords("./stopwords.txt")

def remove_stopwords(segmented_text):
     #字符串转为列表
     segmented_list = segmented_text.split(',')#根据逗号分隔符，将字符串转为列表
     #去停用词
     words_without_stopwords = []
     for w in segmented_list:
         if w not in stopwords:
             words_without_stopwords.append(w)
     return words_without_stopwords

In [4]:
# 从文件中读取comments和对应标签
def Get_comments_and_data(path):
    data=pd.read_csv(path)
    # 使用jieba库对comments进行语句切割
    data['Comment'] = data['Comment'].astype(str) # 不加astype可能出错
    data['Comment'] = data['Comment'].apply(word_cut)
    # 使用去停用词函数，去除comments中的停用词
    data['Comment'] = data['Comment'].apply(remove_stopwords)
    # 提取评论及评价
    comments=data['Comment'].values
    labels=data['label'].values
    # 转成列表
    comments=comments.tolist()
    labels=labels.tolist()
    return [comments,labels]
[comments,labels]=Get_comments_and_data("./Origin_Comments/new_big.csv")

NameError: name 'word_cut' is not defined

In [ ]:
# 删除comments中的评论的''以及' '以及'\n'
def Remove_blanks(comments):
    for com in range(len(comments)):
        while ' ' in comments[com]:
            comments[com].remove(' ')
        while '' in comments[com]:
            comments[com].remove('')
        while '\n' in comments[com]:
            comments[com].remove('\n')
Remove_blanks(comments)

comments

np.save("./Comments.npy",comments)

In [22]:
comments=np.load("./Comments_CUT.npy",allow_pickle=True)

In [23]:
comments

array([list(['成本', '低廉', 'PPT', '电影', 'Nico', '生命', '中', '一年', '发生', '事', 'Nico', '歌配', '情节', '倒', '不算', '尴尬', '女猪', '演得', '不错', '电影', '中', '少见', '中老年', '女', '摇滚', '艺人', '形象', '念惜', '过往', '落魄', '潦倒', '观众', '再少', '挑', '居民楼', '草台', '场子', '煽动', '蹦', '哒', '坦然', '接受', '衰老', '变丑', '接受', '遗弃', '儿子', '后悔', '但小破', '巡演', '宅斗', '奸情', '搞', '搞']),
       list(['传记片', '半真半假', '真实', '成功', '一半', '走进', '人物', '内心世界', '这部', '影片', '纪录片', '真实']),
       list(['意大利', '房东', '记得', '嚣张', '岔开', '大腿', '抽着', '烟', '说', '计划', '优雅', '女人', '总', '念错', '名字', '劳拉', '记得', '黯然', '病床', '前', '问起', '孩子', '真', '幸运', '犹太', '经纪人', '记得', '洒脱', '约定', '伊', '维萨', '回来', '就录', '新专辑', '妮可', '举着', '话筒', '录', '生命', '嚣张', '黯然', '声音', '洒脱', '逃离', '布拉格', '人生', '舞台', '死', '光明节', '烛光', '中']),
       ..., list(['额', '历史剧', '爱情', '剧', 'baby', '剧']),
       list(['找', '一票', '俊', '男', '美女', '熟面孔', '演', '一出', '宫', '心计', '两个', '女神', '养眼', '服化', '良心', '剧情', '太', '偶像剧', '玛丽', '圣母', '白莲花', '安妮', '智商', '堪忧', '娱乐性', '不错']),
       list(['集合', '两', '大美女

In [65]:
labels=np.load("./labels.npy")

In [12]:
def shuffle_data(data,labels):
    indices=np.arange(data.shape[0])
    np.random.shuffle(indices)
    data=data[indices]
    labels=labels[indices]
    return [data,labels]

分析com中发现最长156，最短为0

In [66]:
[data,labels]=shuffle_data(data,labels)

In [27]:
def Trans_texts_to_seq(comments):
    global maxlen,max_words
    tokenizer=Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(comments)
    sequences=tokenizer.texts_to_sequences(comments)
    data=pad_sequences(sequences,maxlen=maxlen)
    return [data,tokenizer]

In [28]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [29]:
[data,tokenizer]=Trans_texts_to_seq(comments)

In [30]:
data1==data

array([[ True,  True,  True, ..., False, False, False],
       [ True,  True,  True, ...,  True, False,  True],
       [ True,  True,  True, ...,  True, False,  True],
       ...,
       [ True,  True,  True, ..., False, False, False],
       [ True,  True,  True, ..., False, False,  True],
       [ True,  True,  True, ...,  True,  True, False]])

In [12]:
data1=data

In [14]:
data1

array([[    0,     0,     0, ..., 17953,   203,   203],
       [    0,     0,     0, ...,    25,   217,    38],
       [    0,     0,     0, ...,    39, 20395,    13],
       ...,
       [    0,     0,     0, ...,   278,  5038,   278],
       [    0,     0,     0, ..., 12366,  4517,    10],
       [    0,     0,     0, ...,  6190,    11,   133]])

In [8]:
data.max()

29999

In [9]:
np.save("./data_50000_100.npy",data)

In [10]:
with open('./different_tokenizer/tokenizer_50000_100.pickle', 'wb') as handle: 
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL) 

In [3]:
with open('./different_tokenizer/tokenizer_50000_100.pickle', 'rb') as handle: 
    tokenizer = pickle.load(handle) 

In [4]:
def Get_embedding_matrix(path): # path中填写预训练词典
    global embedding_dim,max_words,maxlen 
    global tokenizer,wv_from_text
    global index
    wv_from_text = gensim.models.KeyedVectors.load(path, mmap='r')
    # 获得所有词
    vocab = wv_from_text.key_to_index
    word_index=tokenizer.word_index
    embedding_matrix=np.zeros((max_words,embedding_dim))
    for word,i in word_index.items():
        if i<max_words:
            idx=vocab.get(word)# 看是否在字典中
            if idx is not None:# 若找不到索引，则默认为0
                embedding_vector=wv_from_text.get_vector(word)
                embedding_matrix[i]=embedding_vector
            else:
                embedding_matrix[i]=np.random.uniform(size=(maxlen,))
    return embedding_matrix
Embedding_matrix=Get_embedding_matrix("./Pretrain_dic/tencent-ailab-embedding-zh-d100-v0.2.0.bin")

In [5]:
Embedding_matrix.shape

(50000, 100)

In [7]:
Embedding_matrix[0]=np.random.uniform(size=(maxlen,))

In [6]:
Embedding_matrix[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [8]:
np.save("./Embedding/Embedding_matrix_50000_100_big.npy",Embedding_matrix)

In [3]:
Embedding_matrix=np.load("./Embedding/Embedding_matrix_200.npy")

In [8]:
from keras import layers
from keras import models
def Build_model():
    model=models.Sequential()
    model.add(layers.Embedding(max_words,embedding_dim,input_length=maxlen))
    model.add(SeqSelfAttention(64,history_only=True,name='Attention'))
    model.add(layers.LSTM(64))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1,activation='sigmoid'))
    model.layers[0].set_weights([Embedding_matrix])
    model.layers[0].trainable=False
    model.summary()
    return model

np.save("./labels.npy",labels)

In [ ]:
import keras
model=Build_model()
model.compile(optimizer='rmsprop',
             loss='binary_crossentropy',
             metrics=['acc'])
history=model.fit(data,
          labels,
          epochs=20,
          batch_size=128,
          validation_split=0.2)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 200)          10000000  
_________________________________________________________________
Attention (SeqSelfAttention) (None, None, 200)         25729     
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                67840     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 10,093,634
Trainable params: 93,634
Non-trainable params: 10,000,000
_________________________________________________________________
Epoch 1/20
6250/6250 [==============================] - 2196s 351ms/step - loss: 0.5997 - acc: 0.6695 -

In [18]:
len(tokenizer.word_index)

321713

def K_fold_validation(data,labels):
    k=5 # 5折交叉验证
    num_samples=len(data)//k
    shuffle_data(data,labels)
    validation_scores=[]
    num_epochs=10
    for fold in range(k):
        val_data=data[num_samples*fold:num_samples*(fold+1)]
        val_labels=labels[num_samples*fold:num_samples*(fold+1)]
        train_data=np.concatenate((data[:num_samples*fold],data[num_samples*(fold+1):]))
        train_labels=np.concatenate((labels[:num_samples*fold],labels[num_samples*(fold+1):]))
        model=Build_model()
        model.compile(optimizer='rmsprop',
             loss='binary_crossentropy',
             metrics=['acc'])
        model.fit(train_data,train_labels,epochs=num_epochs,batch_size=256)
        loss,accuracy=model.evaluate(val_data,val_labels)
        validation_scores.append(accuracy)
    return sum(validation_scores)/k

scores=K_fold_validation(data,labels)
print(scores)

 model=Build_model()
model.compile(optimizer='rmsprop',
             loss='binary_crossentropy',
             metrics=['acc'])

history=model.fit(x_train,y_train,
                epochs=10,
                 batch_size=128,
                 validation_split=0.2)
import matplotlib.pyplot as plt

history_dict=history.history
loss_values=history_dict['acc']
val_loss_values=history_dict['val_acc']

epoches=range(1,len(loss_values)+1)

plt.plot(epoches,loss_values,'bo',label='Training acc')
plt.plot(epoches,val_loss_values,'b',label='Validation acc')
plt.title('Training and Validation Acc')
plt.xlabel('Epoches')
plt.ylabel('Acc')

plt.legend()
plt.show()